In [1]:
import numpy as np

In [248]:
class Matrix:
    
    def __init__(self, a: np.array = None):
        self.nzir = {}
        self.nzic = {}
        self.es = {}
        self.shape = None
        
        if a is not None:
            n, m = self.shape = a.shape
            for i in range(n):
                for j in range(m):
                    self[i, j] = a[i, j]

    def __setitem__(self, k, v):
        i, j = k
        if not v:
            return
        
        if i not in self.nzir:
            self.nzir[i] = {j}
        else:
            self.nzir[i].add(j)
            
        if j not in self.nzic:
            self.nzic[j] = {i}
        else:
            self.nzic[j].add(i)
            
        self.es[i, j] = v
        
    def __getitem__(self, k):
        if k not in self.es: return 0.
        return self.es[k]
        
    def __mul__(self, x):
        if isinstance(x, float) or isinstance(x, int):
            if not x:
                return Matrix()
            self2 = self.copy()
            for k in self2.es:
                self2.es[k] *= x
            return self2
        
        y = np.zeros(x.shape)
        for i in range(y.shape[1]):
            for r in self.nzir:
                s = 0.
                for c in self.nzic[r]:
                    s += self.es[r, c]*x[c, i]
                y[r, i] = s
        return y
    
    def __rmul__(self, x):
        if isinstance(x, float) or isinstance(x, int):
            if not x:
                return Matrix()
            self2 = self.copy()
            for k in self2.es:
                self2.es[k] *= x
            return self2
        
        y = np.zeros(x.shape)
        for i in range(y.shape[0]):
            for c in self.nzic:
                s = 0.
                for r in self.nzic[c]:
                    s += x[i, r]*self.es[r, c]
                y[i, c] = s
        return y
    
    def copy(self):
        A_ = Matrix()
        A_.nzir = self.nzir.copy()
        A_.nzic = self.nzic.copy()
        A_.es = self.es.copy()
        A_.shape = self.shape
        return A_
    
    def transpose(self):
        A_ = Matrix()
        A_.nzir = self.nzic.copy()
        A_.nzic = self.nzir.copy()
        A_.shape = self.shape
        A_.es = {(j, i): v for (i, j), v in self.es.items()}
        return A_
    
    mul = __mul__
    rmul = __rmul__

The following method minimises $f = x^T A x/2 - b^T x + c$.

In [249]:
def cg_test(x_0, A, b):
    n = len(x_0)
    x = [x_0]
    v = [A.mul(x[0])-b]
    d = [v[0]]
    for i in range(n):
        x.append(
            x[i] - d[i].T@(A.mul(x[i])-b)/(d[i].T@A.mul(d[i])) * d[i]
        )
        v.append(
            A.mul(x[i+1])-b
        )
        d.append(
            v[i+1] + v[i+1].T@v[i+1]/(v[i].T@v[i]) * d[i]
        )
    return x[-1]

In [250]:
A = Matrix(np.array([[1]]))
A.rmul(np.array([[3]]))

array([[3.]])

In [251]:
cg_test(np.array([[1]]), Matrix(np.array([[2]])), np.array([[1]]))

array([[0.5]])

In [252]:
a = np.array([[1, 2], [3, 4]])
A = Matrix(a)
cg_test(np.array([[0], [1]]), A, np.array([[-5], [-6]]))

array([[ 4.68773467],
       [-3.5175219 ]])

To minimise $g = \lvert Ax-b \rvert$, sufficient to do that to $\widehat g = {\lvert Ax-b \rvert}^2$, so the function that computes the answer for the task can be like

In [254]:
def cg_test2(x_0, A, b):
    return cg_test(x_0, Matrix(2*A.transpose().mul(A)), 2*A.mul(b))

In [255]:
a = np.array([[1, 2], [3, 4]])
A = Matrix(a)
cg_test2(np.array([[0], [1]]), A, np.array([[-5], [-6]]))

array([[ 51.5],
       [-38. ]])